# Delete all old results

In [38]:
import os
import glob
from babel.numbers import format_decimal

# Define the pattern to match all .csv files
csv_files = glob.glob("*.csv")

# Loop through the list of files and delete each one
for file in csv_files:
    os.remove(file)
    print(f"Deleted: {file}")

print("All .csv files in the current directory have been deleted.")

Deleted: final stocks.csv
All .csv files in the current directory have been deleted.


# Download Upcoming earning and Bulk-order on stocks from NSE

In [39]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import glob  
from datetime import datetime, timedelta

current_date = datetime.now()
formatted_date = current_date.strftime("%d-%b-%Y")


def getCsvFileName():
    # Get the current date
    current_date = datetime.now()

    # Format the date as 'dd-MMM-yyyy'
    formatted_date = current_date.strftime("%d-%b-%Y")

    # Create the new filename with the current date
    filename = f"CF-Event-equities-{formatted_date}.csv"
    return filename

def getBulkCsvFileName():
    today = datetime.today()
    
    # Calculate the start and end dates
    start_date = today - timedelta(days=7)
    end_date = today
    
    # Format the filename
    file_name = f"Bulk-Deals-{start_date.strftime('%d-%m-%Y')}-to-{end_date.strftime('%d-%m-%Y')}.csv"
    return file_name

def download_pdf(url ,download_click):
    # Initialize the WebDriver using WebDriver Manager
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    
  
    try:
        # Wait for the download button to be clickable
        wait = WebDriverWait(driver, 20)
        download_button = wait.until(EC.element_to_be_clickable((By.ID,download_click)))

        time.sleep(5)
        # Click the download button
        download_button.click()
        
        # Wait for the file to download
        time.sleep(5)  # Adjust sleep time as needed based on download speed

        # Assuming the file is downloaded to the default download directory
        download_dir = os.path.expanduser('~/Downloads')
        downloaded_file_path = os.path.join(download_dir, 'Corporate Filings Event Calendar.csv')

        # Check if the file was downloaded successfully
        # if os.path.exists(downloaded_file_path):
        #     print(f"File downloaded successfully: {downloaded_file_path}")
        # else:
        #     print("File download failed.")
    finally:
        # Close the WebDriver
        driver.quit()

filename=getCsvFileName()
download_dir = os.path.expanduser('~/Downloads')
downloaded_file_path = os.path.join(download_dir, filename)

print(filename)
#Download earning dates
for i in range(1, 10):  
    if os.path.exists(downloaded_file_path):
        print(f"File downloaded successfully: {downloaded_file_path}")
        break     
    download_pdf('https://www.nseindia.com/companies-listing/corporate-filings-event-calendar',"","CFeventCalendar-download")

# Load the CSV into a DataFrame
earnings_df = pd.read_csv(downloaded_file_path)
earnings_df.columns = earnings_df.columns.str.strip().str.replace('\n', '', regex=False)
earnings_df = earnings_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(earnings_df.head)

downloaded_file_path = os.path.join(download_dir, getBulkCsvFileName())

print(filename)
# Download earning dates
for i in range(1, 10):  
    if os.path.exists(downloaded_file_path):
        print(f"File downloaded successfully: {downloaded_file_path}")
        break    
    download_pdf("https://www.nseindia.com/report-detail/display-bulk-and-block-deals","",'HistBulkBlockDeals-download')


# Load the CSV into a DataFrame
bulk_deals_df = pd.read_csv(downloaded_file_path)
bulk_deals_df.columns = bulk_deals_df.columns.str.strip().str.replace('\n', '', regex=False)
bulk_deals_df = bulk_deals_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

print(bulk_deals_df)


CF-Event-equities-23-Jul-2024.csv
File downloaded successfully: /Users/satish.dappure/Downloads/CF-Event-equities-23-Jul-2024.csv
<bound method NDFrame.head of          SYMBOL                                           COMPANY  \
0       ABSLAMC                 Aditya Birla Sun Life AMC Limited   
1     AURIONPRO                       Aurionpro Solutions Limited   
2      AXISBANK                                 Axis Bank Limited   
3    BAJAJFINSV                             Bajaj Finserv Limited   
4    BAJAJHLDNG               Bajaj Holdings & Investment Limited   
..          ...                                               ...   
499    GILLETTE                            Gillette India Limited   
500    GILLETTE                            Gillette India Limited   
501        PGHH  Procter & Gamble Hygiene and Health Care Limited   
502        PGHH  Procter & Gamble Hygiene and Health Care Limited   
503  JAIPURKURT                          Nandani Creation Limited   

           

/var/folders/1f/y_fy20hs4fg7s0vqkf96q1x00000gp/T/ipykernel_4569/4164697061.py:86: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  earnings_df = earnings_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/1f/y_fy20hs4fg7s0vqkf96q1x00000gp/T/ipykernel_4569/4164697061.py:104: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  bulk_deals_df = bulk_deals_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [40]:
# Rename columns to match
bulk_deals_df.rename(columns={'Symbol':'SYMBOL'}, inplace=True)
bulk_deals_df.rename(columns={'Date':'order date'}, inplace=True)
bulk_deals_df.rename(columns={'DATE':'Earning date'}, inplace=True)
bulk_deals_df.rename(columns={'Trade Price / Wght. Avg. Price':'Price'}, inplace=True)

# Rename columns to match
bulk_deals_df.rename(columns={'Buy / Sell':'BUY_SELL'}, inplace=True)


In [41]:
# # Add a helper column for sorting and grouping
# bulk_deals_df['Helper'] = bulk_deals_df.apply(lambda x: (x['Date'], x['SYMBOL'], x['Client Name'], x['Quantity Traded']), axis=1)

# # Group by helper column and filter out the groups with both buy and sell
# grouped = bulk_deals_df.groupby('Helper').filter(lambda x: not ('BUY' in x['BUY_SELL'].values and 'SELL' in x['BUY_SELL'].values))

# # Drop the helper column
# bulk_deals_df = grouped.drop(columns=['Helper'])

# bulk_deals_df.to_csv("remove_dep.csv")
# # Print the resulting DataFrame
# print(bulk_deals_df)

In [42]:
# Merge DataFrames on 'SYMBOL'
merged_df = pd.merge(earnings_df, bulk_deals_df, on='SYMBOL', how='inner')
merged_df.drop(columns=['DETAILS','Security Name','Remarks','PURPOSE'], inplace=True)
merged_df['Quantity Traded'] = merged_df['Quantity Traded'].str.replace(',', '').astype(int)
merged_df['Price'] = merged_df['Price'].str.replace(',', '').astype(float)

merged_df['Quantity Traded'] = merged_df.apply(
    lambda row: -row['Quantity Traded'] if row['BUY_SELL'] == 'SELL' else row['Quantity Traded'],
    axis=1
)
merged_df.to_csv('bulk_deals_with_upcoming.csv', index=False)


# client level orders

In [43]:
# Group by 'SYMBOL' and 'Client Name' and aggregate the values
result_df = merged_df.groupby(['SYMBOL', 'Client Name']).agg({
    'Quantity Traded': 'sum',
    'Price': 'mean'
}).reset_index()

client_level_group = result_df[result_df['Quantity Traded'] != 0]

client_level_group.to_csv("client level.csv")

print(client_level_group)

       SYMBOL                                        Client Name  \
0      AKSHAR                        SIVAPRASAD SIVARAMAKRISHNAN   
3   CYBERTECH                               HRTI PRIVATE LIMITED   
4   ELGIEQUIP            AL MEHWAR COMMERCIAL INVESTMENTS L.L.C.   
7        MTNL                               AAKRAYA RESEARCH LLP   
8        MTNL                              BONANZA PORTFOLIO LTD   
10       MTNL            DIPAN MEHTA COMMODITIES PRIVATE LIMITED   
11       MTNL                        F3 ADVISORS PRIVATE LIMITED   
13       MTNL                               HRTI PRIVATE LIMITED   
14       MTNL                             JAINAM BROKING LIMITED   
15       MTNL                    JSI INVESTMENTS PRIVATE LIMITED   
16       MTNL                               KAMLESH BABALAL SHAH   
17       MTNL             MANSI SHARE AND STOCK ADVISORS PVT LTD   
19       MTNL  MULTIPLIER SHARE & STOCK ADVISORS PRIVATE LIMITED   
21       MTNL                 NAMAN SECURITIES &

# Stock level bulk orders

In [44]:
# Group by 'SYMBOL' and 'Client Name' and aggregate the values
stock_level_group = client_level_group.groupby(['SYMBOL']).agg({
    'Quantity Traded': 'sum',
    'Price': 'mean',
    'Client Name': lambda x: list(x)

   
}).reset_index()

total_value= stock_level_group["Quantity Traded"]* stock_level_group["Price"]
stock_level_group.insert(3, 'Total Amount', total_value)
stock_level_group['Total Amount'] = stock_level_group['Total Amount'].astype(int)


# Custom function to format numbers in Indian style using babel
def format_number_in_indian_style(number):
    return format_decimal(number, locale='en_IN')

stock_level_group['Total Amount'] = stock_level_group['Total Amount'].apply(format_number_in_indian_style)

stock_level_group.to_csv("final stocks.csv")





# Bulk SOLD stocks, whose Earning is in this Week

In [48]:
stock_with_negative_move=stock_level_group[stock_level_group["Quantity Traded"]<0]
print(stock_with_negative_move.loc[:,["SYMBOL","Quantity Traded"]])

print("\n\n Bulk SOLD stocks, whose Earning is in this Week")
listOfStocks=stock_with_negative_move["SYMBOL"].tolist()

for each in listOfStocks:
    print("\nhttps://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3A"+each)
# for symbol in stock_with_positive_move['SYMBOL'].to_string(index=False):
#     print(symbol)


      SYMBOL  Quantity Traded
0     AKSHAR         -3084884
2  ELGIEQUIP         -2297777
3       MTNL         -6691976
5       SHAH         -2100000
6  STYLAMIND           -98734


 Bulk SOLD stocks, whose Earning is in this Week

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3AAKSHAR

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3AELGIEQUIP

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3AMTNL

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3ASHAH

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3ASTYLAMIND


# Bulk Bought stocks, whose Earning is in this Week

In [46]:

stock_with_positive_move=stock_level_group[stock_level_group["Quantity Traded"]>0]
print(stock_with_positive_move.loc[:,["SYMBOL","Quantity Traded"]])

print("\n\n Bulk Bought stocks, whose Earning is in this Week")

listOfStocks=stock_with_positive_move["SYMBOL"].tolist()

for each in listOfStocks:
    print("\nhttps://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3A"+each)
# for symbol in stock_with_positive_move['SYMBOL'].to_string(index=False):
#     print(symbol)


      SYMBOL  Quantity Traded
1  CYBERTECH             5750
4   ONMOBILE            45826
7       TTML            92570


 Bulk Bought stocks, whose Earning is in this Week

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3ACYBERTECH

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3AONMOBILE

https://in.tradingview.com/chart/YRlxk2JY/?symbol=NSE%3ATTML
